In [1]:
import os
# Suppress TensorFlow C++ backend logs:
# '0' = all logs, '1' = filter out INFO, 
# '2' = filter out INFO + WARNING, 
# '3' = filter out all except ERROR
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import warnings
# Suppress all Python warnings (e.g., deprecation warnings)
warnings.filterwarnings('ignore')

import tensorflow as tf
# Suppress TensorFlow Python-level logs (only show errors)
tf.get_logger().setLevel('ERROR')

In [2]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# ========================================
# LOAD SCALER PARAMETERS (MEAN & SCALE)
# ========================================
# These were saved earlier when training, so that
# test/validation data can be normalized the same way
scaler_mean = np.load("scaler_mean_combined.npy")   # mean of training data
scaler_scale = np.load("scaler_scale_combined.npy") # std. deviation of training data

# ========================================
# RECREATE SCALER WITH SAVED PARAMETERS
# ========================================
scaler = StandardScaler()
scaler.mean_ = scaler_mean              # set mean manually
scaler.scale_ = scaler_scale            # set std manually
scaler.var_ = scaler_scale ** 2         # variance = std^2

# ========================================
# LOAD TRAINED MODELS
# ========================================
from tensorflow.keras.models import load_model

# Load different final models trained earlier
residual_cnn_model   = load_model("final_combined_cnn_model.h5")       # Residual CNN model
cnn_gru_model        = load_model("final_combined_cnn_gru_model.h5")   # CNN + GRU hybrid
pure_cnn_model       = load_model("final_pure_cnn_model.h5")           # Pure deep CNN
cnn_transormer_model = load_model("final_cnn_transformer_model.h5")    # CNN + Transformer hybrid

In [3]:
import pandas as pd

# ========================================
# LOAD PROCESSED BBH SIGNAL DATA
# ========================================
# Each row in processed.csv = one BBH event (strain time series, 4s long)
data_BBH_df = pd.read_csv("./BBH/processed.csv", header=None)

# Convert DataFrame to NumPy array of floats
X = data_BBH_df.values.astype(float)

# Print number of BBH events loaded
print(len(data_BBH_df.index))

# ========================================
# CREATE LABELS
# ========================================
# Label '1' for all rows (since these are real GW signals, not noise)
y = [1 for i in range(len(data_BBH_df.index))]

# ========================================
# APPLY SCALING
# ========================================
# Normalize data using the previously loaded StandardScaler
X = scaler.transform(X)

# ========================================
# RESHAPE FOR CNN INPUT
# ========================================
# Add channel dimension at the end: (n_samples, n_timesteps, 1)
# Example: (N, 16384) → (N, 16384, 1)
X = np.expand_dims(X, axis=-1)

# Print shape to confirm
print(X.shape)

FileNotFoundError: [Errno 2] No such file or directory: './BBH/processed.csv'

In [ ]:
# ========================================
# PREDICT WITH RESIDUAL CNN MODEL
# ========================================
# Run model prediction on input data X
# Output = probabilities between 0 and 1
pred_residual_cnn = residual_cnn_model.predict(X)
# Convert probabilities to binary class predictions
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
y_pred = (pred_residual_cnn.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count how many predictions match the true labels (y)
correct = np.sum(y_pred == y)
# Compute accuracy in percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by Residual CNN model: {correct}")
print(f"Accuracy of Residual CNN model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH CNN + GRU MODEL
# ========================================
# Run model prediction on input data X
# Output = probabilities between 0 and 1
pred_cnn_gru = cnn_gru_model.predict(X)
# Convert probabilities to binary predictions:
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
y_pred = (pred_cnn_gru.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count correct predictions
correct = np.sum(y_pred == y)
# Calculate accuracy in percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by CNN + GRU model: {correct}")
print(f"Accuracy of CNN + GRU model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH PURE CNN MODEL
# ========================================
# Run model prediction on input data X
# Model outputs probabilities between 0 and 1
pred_pure_cnn = pure_cnn_model.predict(X)
# Apply classification threshold:
# > 0.5 → predict class 1 (signal), <= 0.5 → predict class 0 (noise)
y_pred = (pred_pure_cnn.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count number of correct predictions
correct = np.sum(y_pred == y)
# Compute accuracy as a percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by Pure CNN model: {correct}")
print(f"Accuracy of Pure CNN model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH CNN + TRANSFORMER MODEL
# ========================================
# Run model prediction on input data X
# Output = probabilities (between 0 and 1)
pred_cnn_transormer = cnn_transormer_model.predict(X)
# Convert probabilities into binary predictions
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
y_pred = (pred_cnn_transormer.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count number of correct predictions
correct = np.sum(y_pred == y)
# Calculate accuracy in percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions by CNN + Transormfer model: {correct}")
print(f"Accuracy of CNN + Transormfer model: {acc:.2f}%")

In [ ]:
#
#
# Ensemble technique implementation
#
#

# ========================================
# AVERAGE PREDICTIONS FROM ALL MODELS
# ========================================
# Each model outputs probabilities between 0 and 1
# We take the average across all four models
ensemble_pred = (pred_residual_cnn.ravel() + pred_cnn_gru.ravel() + pred_pure_cnn.ravel() + pred_cnn_transormer.ravel()) / 4
# ========================================
# APPLY CLASSIFICATION THRESHOLD
# ========================================
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
final_pred = (ensemble_pred.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE ENSEMBLE PERFORMANCE
# ========================================
# Count how many ensemble predictions match ground truth
correct_final_pred = np.sum(final_pred == y)
# Compute accuracy percentage
acc_final = (correct_final_pred / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original gravitational data samples: {len(y)}")
print(f"No. of correct predictions: {correct_final_pred}")
print(f"Accuracy: {acc_final:.2f}%")

In [ ]:
import pandas as pd

# ========================================
# LOAD UNSEEN NOISE DATA
# ========================================
# Each row in original-noise-unseen.csv = one unseen noise segment
data_BBH_df = pd.read_csv("./BBH/original-noise-unseen.csv", header=None)
# Convert DataFrame to NumPy array of floats
X = data_BBH_df.values.astype(float)
# Print how many unseen noise samples are loaded
print(len(data_BBH_df.index))
# ========================================
# CREATE LABELS
# ========================================
# Label '0' for all rows (since these are pure noise, not signals)
y = [0 for i in range(len(data_BBH_df.index))]
# ========================================
# APPLY SAME NORMALIZATION AS TRAINING
# ========================================
# Use the previously loaded StandardScaler (scaler)
X = scaler.transform(X)
# ========================================
# RESHAPE FOR CNN INPUT
# ========================================
# Add a channel dimension so shape = (n_samples, n_timesteps, 1)
# Example: (N, 16384) → (N, 16384, 1)
X = np.expand_dims(X, axis=-1)
# Print final shape to confirm
print(X.shape)

In [ ]:
# ========================================
# PREDICT WITH RESIDUAL CNN MODEL
# ========================================
# Run model prediction on unseen noise samples (X)
# Model outputs probabilities between 0 and 1
pred_residual_cnn = residual_cnn_model.predict(X)
# Apply threshold:
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
y_pred = (pred_residual_cnn.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count how many predictions match the true labels (y = all 0s for noise)
correct = np.sum(y_pred == y)
# Compute accuracy percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by Residual CNN model: {correct}")
print(f"Accuracy of Residual CNN model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH CNN + GRU MODEL
# ========================================
# Run model prediction on unseen noise data (X)
# Model outputs probabilities between 0 and 1
pred_cnn_gru = cnn_gru_model.predict(X)
# Apply classification threshold:
# > 0.5 → predict class 1 (signal)
# <= 0.5 → predict class 0 (noise)
y_pred = (pred_cnn_gru.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count how many predictions match the ground truth labels (y = all 0s for noise)
correct = np.sum(y_pred == y)
# Compute accuracy as percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by CNN + GRU model: {correct}")
print(f"Accuracy of CNN + GRU model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH PURE CNN MODEL
# ========================================
# Run model prediction on unseen noise data (X)
# Model outputs probabilities between 0 and 1
pred_pure_cnn = pure_cnn_model.predict(X)
# Apply classification threshold:
# > 0.5 → predict class 1 (signal)
# <= 0.5 → predict class 0 (noise)
y_pred = (pred_pure_cnn.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count how many predictions match the ground truth labels (y = all 0s for noise)
correct = np.sum(y_pred == y)
# Compute accuracy as percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by Pure CNN model: {correct}")
print(f"Accuracy of Pure CNN model: {acc:.2f}%")

In [ ]:
# ========================================
# PREDICT WITH CNN + TRANSFORMER MODEL
# ========================================
# Run model prediction on unseen noise data (X)
# Model outputs probabilities between 0 and 1
pred_cnn_transormer = cnn_transormer_model.predict(X)
# Apply classification threshold:
# > 0.5 → predict class 1 (signal)
# <= 0.5 → predict class 0 (noise)
y_pred = (pred_cnn_transormer.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE PREDICTIONS
# ========================================
# Count how many predictions match the ground truth labels (y = all 0s for noise)
correct = np.sum(y_pred == y)
# Compute accuracy as percentage
acc = (correct / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions by CNN + Transormfer model: {correct}")
print(f"Accuracy of CNN + Transormfer model: {acc:.2f}%")

In [ ]:
#
#
# Ensemble technique implementation
#
#

# ========================================
# AVERAGE PREDICTIONS FROM ALL MODELS
# ========================================
# Each model outputs probabilities between 0 and 1.
# We combine predictions by taking their average.
ensemble_pred = (pred_residual_cnn.ravel() + pred_cnn_gru.ravel() + pred_pure_cnn.ravel() + pred_cnn_transormer.ravel()) / 4
# ========================================
# APPLY CLASSIFICATION THRESHOLD
# ========================================
# Convert averaged probabilities to binary predictions:
# > 0.5 → class 1 (signal), <= 0.5 → class 0 (noise)
final_pred = (ensemble_pred.ravel() > 0.5).astype(int)
# ========================================
# EVALUATE ENSEMBLE PERFORMANCE
# ========================================
# Count how many ensemble predictions match the ground truth labels (y = all 0s here)
correct_final_pred = np.sum(final_pred == y)
# Compute accuracy as a percentage
acc_final = (correct_final_pred / len(y)) * 100

# ========================================
# PRINT RESULTS
# ========================================
print(f"Total no. of original noise data samples: {len(y)}")
print(f"No. of correct predictions: {correct_final_pred}")
print(f"Accuracy: {acc_final:.2f}%")